In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ------------------------------------------------------------
# CONFIGURATION
# ------------------------------------------------------------
# Coordonnées de Montréal (centre)
lat = 45.5017
lon = -73.5673

# Plage historique que tu veux
start_date = "2022-01-01"
end_date = "2025-08-31"  # exemple

# Variables météo horaires que tu veux
variables = [
    "temperature_2m", 
    "precipitation", 
    "windspeed_10m", 
    "winddirection_10m", 
    "relativehumidity_2m", 
    "cloudcover"
]

# L’URL de l’API
base_url = "https://archive-api.open-meteo.com/v1/archive"

params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date,
    "end_date": end_date,
    "hourly": ",".join(variables),
    "timezone": "America/Toronto"  # pour avoir les heures locales
}

# ------------------------------------------------------------
# APPEL API
resp = requests.get(base_url, params=params, verify=False)
resp.raise_for_status()  # lève une erreur si mauvais status code

data = resp.json()

# ------------------------------------------------------------
# TRANSFORMATION DES DONNÉES MÉTÉO
# ------------------------------------------------------------
# On suppose que la réponse contient un champ "hourly" avec les variables
# et un champ "hourly.time" pour les timestamps

df_meteo = pd.DataFrame(data["hourly"])
# Exemple des colonnes attendues : "time", "temperature_2m", "precipitation", etc.

# Séparation date / heure
# time a un format comme "2025-01-01T00:00" ou "2025-01-01T00:00:00"
df_meteo["date"] = pd.to_datetime(df_meteo["time"]).dt.date.astype(str)
df_meteo["heure"] = pd.to_datetime(df_meteo["time"]).dt.strftime("%H:%M:%S")

# On peut renommer les colonnes météo pour plus de clarté
df_meteo = df_meteo.rename(columns={
    "temperature_2m": "temp_c",
    "precipitation": "precip_mm",
    "windspeed_10m": "wind_kmh",
    "relativehumidity_2m": "humidité_pct",
    "cloudcover": "cloud_pct"
})

# Sélection des colonnes utiles
colonnes_use = ["date", "heure"] + [
    col for col in df_meteo.columns if col in ["temp_c", "precip_mm", "wind_kmh", "humidité_pct", "cloud_pct"]
]
df_meteo = df_meteo[colonnes_use]
# noms de colonnes "humidité" -> "humidite_pct")
df_meteo = df_meteo.rename(columns={
    "humidité_pct": "humidite_pct"
})
# ------------------------------------------------------------
# FUSION AVEC TON DATASET FINAL
# ------------------------------------------------------------
# Supposons que tu as déjà df_final (issu de ta fusion vélo + intersection + etc.)
# Assure-toi que dans df_final il y a les colonnes "date" (même format string ou datetime) et "heure"

# Convertir la date / heure de df_final si nécessaire
# Exemple si date et heure sont des chaînes :
# df_final['date'] = pd.to_datetime(df_final['date']).dt.date.astype(str)

# df_final['heure'] = pd.to_datetime(df_final['heure']).dt.strftime("%H:%M:%S")

# Fusion
display(df_meteo.head())
#explore date from when to when
print(df_meteo['date'].min(), df_meteo['date'].max())
df_meteo.info()
#export df_meteo to csv
df_meteo.to_csv("C:/Users/Public/Documents/TELECHARGEMENT/BdeB/A52-Algorithme d'apprentissage supervisés/TP_Modele/meteo_montreal_horaire.csv", index=False)








ModuleNotFoundError: No module named 'requests'